# Genetic analysis of cercarial production (SmBRE x SmLE)


## Aim

This notebook describes the different analysis done to map QTL related to production of cercariae by high and low shedder schistosomes. This work is the extension of the previous work described in [Le Clec'h *et al.* 2019](https://doi.org/10.1186/s13071-019-3741-z). Here two reciprocal crosses have been conducted between the SmLE, high shedder schistosome, and SmBRE, low shedder schistosome.

We analyze the phenotype of the parents, F1s and F2s using different measures (weekly cercariae shed, average cercariae shed) and co-variate (sex). We also analyze how the schistosome phenotype influenced snail traits (hemoglobin rate and phenoloxidase activity). We finally mapped the parasite phenotype on the reference genome. 



## Environment and data

In [ ]:
conda env create -f .env/env.yml

The cell below must be run each time a new Jupyter session is run.

In [1]:
# Activate the environment
source $(sed "s,/bin/conda,," <<<$CONDA_EXE)/etc/profile.d/conda.sh
conda activate cerc_prod

# Remove potential variable interferences
export PERL5LIB=""
export PYTHONNOUSERSITE=1

(cerc_prod) 

: 1

The cell below must be run only once at the time of the environment creation.

In [ ]:
# Installing needed R packages
Rscript ".env/R package dependencies.R"

### Phenotypic data

The phenotypic data for the F0 parental population, the F1s and F2s is available on [Zenodo](https://doi.org/10.5281/zenodo.4383248). Data is already present in the repository but can be downloaded if needed.

In [ ]:
# Data directory
pdir="data/phenotypes"
[[ ! -d "$pdir" ]] && mkdir -p "$pdir"

# Download files
wget -q -P "$pdir" https://zenodo.org/record/4383248/files/F0_parental_populations.csv
wget -q -P "$pdir" https://zenodo.org/record/4383248/files/F1.csv
wget -q -P "$pdir" https://zenodo.org/record/4383248/files/F2.csv
wget -q -P "$pdir" https://zenodo.org/record/4383248/files/sex.tsv

### Sequencing data

This step downloads the fastq files of the different samples from the SRA repository.

In [ ]:
# Data directory
ldir="data/libraries"
[[ ! -d "$ldir" ]] && mkdir -p "$ldir"

# Bioproject
bioproject=PRJNA667697

# Download related information to data project
wget -q -O runinfo "http://trace.ncbi.nlm.nih.gov/Traces/sra/sra.cgi?save=efetch&rettype=runinfo&db=sra&term=${bioproject}"

# Field of interest (library name and weblink)
fdn=$(head -n 1 runinfo | tr "," "\n" | grep -w -n "LibraryName" | cut -d ":" -f 1)
fdr=$(head -n 1 runinfo | tr "," "\n" | grep -w -n "Run" | cut -d ":" -f 1)

# Download fastq files
while read line
do
    # Filename and download link
    fln=$(cut -d "," -f $fdn <<<$line)
    run=$(cut -d "," -f $fdr <<<$line)
    
    # Download
    echo "$fln"
    [[ ! -d "$ldir/$fln/" ]] && mkdir -p "$ldir/$fln/"
    fastq-dump -O "$ldir/$fln/" --split-files "$run"
    
    mv "$ldir/$fln/${run}_1.fastq" "$ldir/$fln/${fln}_R1.fastq"
    mv "$ldir/$fln/${run}_2.fastq" "$ldir/$fln/${fln}_R2.fastq"
        
done < <(tail -n +2 runinfo | sed "/^$/d")

# Compress files
pigz "$ldir/"*/*

rm runinfo

### Genome data

The genome data is downloaded from the [WormBase ParaSite](https://parasite.wormbase.org). We use the data from the version 14 (WBPS14). The data is then indexed for the different tools used.

In [ ]:
gdir="data/genome"
[[ ! -d "$gdir" ]] && mkdir -p "$gdir"

# Download and unzip data
wget -P "$gdir" ftp://ftp.ebi.ac.uk/pub/databases/wormbase/parasite/releases/WBPS14/species/schistosoma_mansoni/PRJEA36577/schistosoma_mansoni.PRJEA36577.WBPS14.genomic.fa.gz
pigz -d "$gdir/schistosoma_mansoni.PRJEA36577.WBPS14.genomic.fa.gz"

# Preparing indices
bwa index "$gdir/schistosoma_mansoni.PRJEA36577.WBPS14.genomic.fa"
samtools faidx "$gdir/schistosoma_mansoni.PRJEA36577.WBPS14.genomic.fa"
gatk CreateSequenceDictionary -R "$gdir/schistosoma_mansoni.PRJEA36577.WBPS14.genomic.fa"

### snpEff database

To annotate the functional consequences of the variants in the VCF file, we use the snpEff tool. Because the reference genome is recent, we have to build a snpEff database in order to annotate the variants. 

In [ ]:
# Create snpEff folder
sedir="$gdir/snpEff/"
mkdir "$sedir"

# Download the S. mansoni genome annotation
wget -P "$gdir" ftp://ftp.ebi.ac.uk/pub/databases/wormbase/parasite/releases/WBPS14/species/schistosoma_mansoni/PRJEA36577/schistosoma_mansoni.PRJEA36577.WBPS14.annotations.gff3.gz
pigz -d "$gdir/schistosoma_mansoni.PRJEA36577.WBPS14.annotations.gff3.gz"

# Convert gff to gtf to make it easier for snpEff
gffread "$gdir/schistosoma_mansoni.PRJEA36577.WBPS14.annotations.gff3" -T -o "$gdir/schistosoma_mansoni.PRJEA36577.WBPS14.annotations.gtf"

# Custom snpEff config file
cp "$CONDA_PREFIX"/share/snpeff-4.3.1t-4/snpEff.config "$sedir"

## Identify mitochondrial chromosome
mito=$(grep -i ">.*mito.*" "$gdir/schistosoma_mansoni.PRJEA36577.WBPS14.genomic.fa" | sed -r "s/>(.*) .*/\1/")

##  Add schistosome genome and specify mitochondrial chromosome
sed -i -r "s|data.dir = ./data/|data.dir = .| ; s|(# Ebola virus)|# S. mansoni\nsmansoni.genome: Schistosoma_mansoni\n\tsmansoni.$mito.codonTable : Trematode_Mitochondrial\n\n\1|" "$sedir"/snpEff.config

# Create the mandatory folder and link to the needed files
mkdir "$sedir"/genomes
ln -s "$PWD/$gdir/schistosoma_mansoni.PRJEA36577.WBPS14.genomic.fa" "$PWD/$sedir/genomes/smansoni.fa"

mkdir "$sedir"/smansoni
ln -s "$PWD/$gdir/schistosoma_mansoni.PRJEA36577.WBPS14.annotations.gtf" "$PWD/$sedir/smansoni/genes.gtf"

# Generate the database
snpEff build -v -c data/genome/snpEff/snpEff.config smansoni

### Known variants

List of variants from the SmLE x SmHR crosses that segregated in a Mendelian fashion in F1 ([Valentim *et al.* 2013](https://doi.org/10.1126/science.1243106)). This list was generated from alignments using the v5 genome. This has been lifted over to the latest genome using the [flo pipeline](https://github.com/wurmlab/flo/tree/727f10b2b1c57a0514835d302d7f6345d3a34ffb).

In [ ]:
# Index file
gatk IndexFeatureFile -I "$gdir/sm_dbSNP_v7.vcf"

## Sequencing data processing

### Snakemake pipeline

The snakemake pipeline performs the following steps:
* alignment of each library against the reference genome
* marking duplicates
* base quality recalibration
* generating alignment statistics
* outputing Z chromosome read depth for each library
* determining sex of each individual
* calling variants (gVCF) for each library
* combining gVCF of all libraries
* genotyping all samples per contig
* merging all the contigs to generate final VCF file

The VCF file produced will then be used to identify possible QTLs link to the cercarial production.

In [ ]:
snakemake --snakefile snakefile --cluster "qsub -V -cwd -o status -j y -r y -pe smp 12 -S /bin/bash" --jobs 384 -w 300

### Variant annotation

Functional consequences of variants is annotated to the VCF file using snpEff.

In [ ]:
snpEff -v -c "$sedir/snpEff.config" smansoni "data/calling/cerc_prod.vcf.gz" | bgzip -c -@ $(nproc) > "data/calling/cerc_prod_snpEff.vcf.gz"

# Index the VCF file
tabix "data/calling/cerc_prod_snpEff.vcf.gz"

In [11]:
# Move file in the right folder
mv -t "data/calling/" snpEff_summary.* snpEff_genes.txt

(cerc_prod) (cerc_prod) 

: 1

### Selection of informative sites

Because we used two different types of library preparation, exome capture (F1 and F2) and whole genome sequence (parents), most of the sites identified during the variant calling are uninformative (only identified in parents). To speed up analysis, only sites that have less than 50% of missing data are retained.

In [3]:
gatk SelectVariants \
        -V "data/calling/cerc_prod_snpEff.vcf.gz" \
        --max-nocall-fraction 0.5 \
        -O "data/calling/cerc_prod_snpEff_reduced.vcf.gz"

Using GATK jar /master/fcheval/local/pckg/python/miniconda3/envs/cerc_prod/share/gatk4-4.1.7.0-0/gatk-package-4.1.7.0-local.jar
Running:
    java -Dsamjdk.use_async_io_read_samtools=false -Dsamjdk.use_async_io_write_samtools=true -Dsamjdk.use_async_io_write_tribble=false -Dsamjdk.compression_level=2 -jar /master/fcheval/local/pckg/python/miniconda3/envs/cerc_prod/share/gatk4-4.1.7.0-0/gatk-package-4.1.7.0-local.jar SelectVariants -V data/calling/cerc_prod_snpEff.vcf.gz --max-nocall-fraction 0.5 -O data/calling/cerc_prod_snpEff_reduced.vcf.gz
23:02:48.844 INFO  NativeLibraryLoader - Loading libgkl_compression.so from jar:file:/master/fcheval/local/pckg/python/miniconda3/envs/cerc_prod/share/gatk4-4.1.7.0-0/gatk-package-4.1.7.0-local.jar!/com/intel/gkl/native/libgkl_compression.so
Jun 21, 2020 11:02:49 PM shaded.cloud_nio.com.google.auth.oauth2.ComputeEngineCredentials runningOnComputeEngine
INFO: Failed to detect whether we are running on Google Compute Engine.
23:02:49.906 INFO  Select

23:11:00.103 INFO  ProgressMeter -    SM_V7_ZW:22063135              8.2               1266000         155147.1
23:11:10.374 INFO  ProgressMeter -    SM_V7_ZW:25195527              8.3               1293000         155200.0
23:11:20.639 INFO  ProgressMeter -    SM_V7_ZW:28354401              8.5               1319000         155135.1
23:11:30.656 INFO  ProgressMeter -    SM_V7_ZW:31622083              8.7               1345000         155146.7
23:11:40.864 INFO  ProgressMeter -    SM_V7_ZW:35260079              8.8               1372000         155215.0
23:11:51.232 INFO  ProgressMeter -    SM_V7_ZW:39297817              9.0               1399000         155234.9
23:12:01.385 INFO  ProgressMeter -    SM_V7_ZW:42908389              9.2               1426000         155314.6
23:12:11.709 INFO  ProgressMeter -    SM_V7_ZW:45923529              9.4               1453000         155344.0
23:12:21.839 INFO  ProgressMeter -    SM_V7_ZW:48278818              9.5               1479000         1

23:23:34.856 INFO  ProgressMeter -      SM_V7_4:1455892             20.7               3196000         154104.3
23:23:45.185 INFO  ProgressMeter -      SM_V7_4:3765168             20.9               3223000         154126.7
23:23:55.485 INFO  ProgressMeter -      SM_V7_4:5796481             21.1               3250000         154152.5
23:24:05.568 INFO  ProgressMeter -      SM_V7_4:8336418             21.3               3275000         154109.8
23:24:15.778 INFO  ProgressMeter -     SM_V7_4:10930346             21.4               3302000         154146.0
23:24:26.083 INFO  ProgressMeter -     SM_V7_4:13393724             21.6               3329000         154170.3
23:24:36.104 INFO  ProgressMeter -     SM_V7_4:14946468             21.8               3354000         154135.9
23:24:46.481 INFO  ProgressMeter -     SM_V7_4:17419630             21.9               3382000         154197.2
23:24:56.766 INFO  ProgressMeter -     SM_V7_4:19985892             22.1               3408000         1

: 1

## Phenotype analysis

### *In silico* schistosome sexing

Sex is often an important covariate. We used Z chromosome read depth to determine sex of the cercariae. Sexual determination in schistosome relies on Z and W chromosomes: females are ZW while males are ZZ. We take advantage of the sequencing data to perform an *in silico* sexing by comparing the read depth along the Z chromosome. The Z chromosome carries a Z-linked region (located between 11 and 44 Mb) which never recombines with the W chromosome contrariwise to the rest of the Z chromosome (pseudo-autosomal region). While the Z-linked region in males (ZZ) will display the same read depth as the pseudo-autosomal region, the Z-linked region in females (ZW) will display only half of the read depth of the pseudo-autosomal region. We can therefore determine a read depth ratio between the Z-linked and pseudo-autosomal regions: a ratio around 1 will correspond to a male carrying two Z chromosomes while a ratio around 0.5 will correspond to a female carrying only one Z chromosome.

We compute the read depth at each base of the chromosome Z using SAMtools and BEDtools. We use R to first smooth the read depth data which may show local high variation using the runmed function on 101 contiguous sites, then we keep only sites showing a read depth of more than 5 to finally compute the ratio on Z-linked over Z pseudo-autosomal read depth. We validated this approach by comparing molecular and in silico sexing of some F2s.

In [3]:
# Result folder
rdir="results"
[[ ! -d "$rdir/1-Phenotype" ]] && mkdir -p "$rdir/1-Phenotype"

sex="$rdir/1-Phenotype/sex.tsv"

# Header
echo -e "id\tread_depth\tratio\tsex" > "$sex"

# Add parents
for i in $(find "$ldir" -maxdepth 1 -name S* | sed "s|^.*/||g")
do
    echo -e "$i\t$(cat "$ldir/$i/"*sex | tr "\n" "\t")"
done >> "$sex"

# Add F1s and F2s
for i in $(find "$ldir" -maxdepth 1 -name F* | sed "s|^.*/||g")
do
    echo -e "$i\t$(cat "$ldir/$i/"*sex | tr "\n" "\t")"
done >> "$sex"

### Transmission stage production

We analyze the cercarial production of the two parental populations (i.e. SmLE-H and SmBRE-L) and their progeny (i.e. F1s and F2s). This includes:
* Difference in the average of cercariae produced by SmLE-H and SmBRE-L populations during 4 weeks of the patent period (week 4 to 7 post infection). SmLE-H population is shedding more cercariae than SmBRE-L population. This data are those already published in [Le Clec'h *et al.* 2019](https://doi.org/10.1186/s13071-019-3741-z). 
* Difference in the number of cercariae produced by parental populations (i.e. SmLE-H and SmBRE-L) and progeny (i.e. F1 and F2) measured by week (week 4 to 7 post infection). 
* Distribution of the cercarial production (mean + SE) over the 4 weeks of the patent period for the parents (F0), the F1 parents and all the F2 progeny by rank order.

In [ ]:
Rscript scripts/Fig2_cercarial_shedding_pheno.R

### Impact of parasite gender on the cercarial production

We analyze the impact of the parasite sex on cercarial production for F1s and F2s. 

In [ ]:
Rscript scripts/Supp_Fig1_cercarial_shedding_sex.R

### Correlation between snail traits and cercarial production

We measure to snail traits on the hemolymph of the infected snails: the laccase-like activity and the hemoglobin. Measure were taken at the end of the experiment for the three generations. These measures can then be correlated to the cercarial production.

In [ ]:
Rscript scripts/Supp_Fig3_correlation_PO_Hb_shedding.R

## QTL analysis

The analysis relies on several steps:
* **SNP filtration**: bi-allelic site only, read depth (DP) of 10 at least, genotype quality (GQ) of 30.
* **Conversion of the genotype data in R/qtl format (csvs)**: only alternatively fixed alleles from the parent retained.
* **Single QTL analysis on all the phenotypes** (estimated run time: 30 minutes): sum of cercariae shed by schistosome genotype, average of cercariae shed by schistosome genotype, total phenoloxidase activity of the snail host, hemoglobin rate of the snail host.
* **QTL interaction analysis using reduced set of markers** (estimated run time: 12h).
* **Allele dominance analysis.**

N.B.: Estimated run time indicated is dependent on computer/server specifications. Our analysis ran on a server with 16 CPU (2.53 GHz) and 96 GB of RAM.

In [ ]:
Rscript scripts/Cercariae_intensity_qtl.R
Rscript scripts/Fig4_plot_dominance_chr.R

## Prioritization of candidate genes

### Chromosome level VCF

QTLs were identified on chromosomes 1, 2, 3, 4 and 5. The VCF file needs to be reduced to chromosome intervals to avoid loading unnecessary data and saving time.

In [6]:
gatk SelectVariants \
        -V "data/calling/cerc_prod_snpEff.vcf.gz" \
        -L SM_V7_1:40000000-55000000 \
        -L SM_V7_2:40000000-48130368 \ # End of the chromosome
        -L SM_V7_3:3000000-5500000 \
        -L SM_V7_4:20000000-40000000 \
        -L SM_V7_5:18000000-25000000 \
        -O "data/calling/cerc_prod_snpEff_1-2-3-4-5.vcf.gz"

Using GATK jar /master/fcheval/local/pckg/python/miniconda3/envs/cerc_prod/share/gatk4-4.1.7.0-0/gatk-package-4.1.7.0-local.jar
Running:
    java -Dsamjdk.use_async_io_read_samtools=false -Dsamjdk.use_async_io_write_samtools=true -Dsamjdk.use_async_io_write_tribble=false -Dsamjdk.compression_level=2 -jar /master/fcheval/local/pckg/python/miniconda3/envs/cerc_prod/share/gatk4-4.1.7.0-0/gatk-package-4.1.7.0-local.jar SelectVariants -V data/calling/cerc_prod_snpEff.vcf.gz -L SM_V7_1:40000000-55000000 -L SM_V7_2:40000000-48130368 -L SM_V7_3:3000000-5500000 -L SM_V7_4:20000000-40000000 -L SM_V7_5:18000000-25000000 -O data/calling/cerc_prod_snpEff_1-2-3-4-5.vcf.gz
11:22:45.276 INFO  NativeLibraryLoader - Loading libgkl_compression.so from jar:file:/master/fcheval/local/pckg/python/miniconda3/envs/cerc_prod/share/gatk4-4.1.7.0-0/gatk-package-4.1.7.0-local.jar!/com/intel/gkl/native/libgkl_compression.so
Aug 04, 2020 11:22:45 AM shaded.cloud_nio.com.google.auth.oauth2.ComputeEngineCredentials r

11:30:47.218 INFO  ProgressMeter -     SM_V7_5:20841294              8.0                574000          71586.5
11:30:58.029 INFO  ProgressMeter -     SM_V7_5:21361676              8.2                586000          71475.6
11:31:08.555 INFO  ProgressMeter -     SM_V7_5:21933207              8.4                598000          71411.2
11:31:19.536 INFO  ProgressMeter -     SM_V7_5:23060814              8.6                611000          71403.1
11:31:30.029 INFO  ProgressMeter -     SM_V7_5:24226449              8.7                624000          71462.0
11:31:40.595 INFO  ProgressMeter -     SM_V7_5:24743588              8.9                636000          71396.2
11:31:42.924 INFO  ProgressMeter -     SM_V7_5:24916478              8.9                638874          71407.7
11:31:42.924 INFO  ProgressMeter - Traversal complete. Processed 638874 total variants in 8.9 minutes.
11:31:43.462 INFO  SelectVariants - Shutting down engine
[August 4, 2020 11:31:43 AM CDT] org.broadinstitute.hell

: 1

### Candidate gene analysis

The 5 QTLs identified cover several Mb and tens of genes. To identify the genes most likely involved in cercarial production, we focus on few criteria:
* We refine the QTL regions by determining the chromosomal positions corresponding to a drop of 1.8 LOD from the highest LOD score of each QTL. We expect the genes involved to be relatively close to the peaks.
* We focus on alternative fix alleles in the parents (with a GQ greater than 29 and a DP greater than 5). The phenotype in the parental population being dramatic, we expect to the parents to be homozygous for the alleles involved.
* We take in account mutations and their impacts in each gene. We expect at least on the parental allele to show mutations with important impacts.
* We final take in account know gene expression in late sporocyst and cercariae. We expect the genes to be expressed at these stages.

Using these criteria, we created a formula to generate a score (genoscore) to objectively prioritize candidate genes in each QTL region:

Genoscore = $\sum_{i=1}^{n} (GSa_i \times GSb_i \times IS_i) \over l \times \sqrt(d_peak))$$ \times e \times 100$

The genoscore is computed either by gene or gene’s coding sequence (CDS). For each site i of a gene or CDS, we first compute a genotype score (GS) for each cross a and b by associating a score of 20 if genotypes are alternatively fixed in each cross parent or a score of 5 if fixed in one parent and unknown in the other (any other genotype combination received 0) and an impact score (IS) from the snpEff annotation (low impact or modifier mutation = 1, moderate impact = 5, high impact = 10). We sum the results for the n sites present in the gene or CDS. We weigh this sum using the length l of the gene or the CDS, and the square root of the distance d between the position of the highest LOD score (peak) and the start of the gene or the CDS. This was finally modulated by an expression factor e of 1 if expressed or -1 otherwise. Jupyter notebook and scripts used to obtain gene expression are available on [Github](https://github.com/fdchevalier/Sm_gene_expression).

In [ ]:
Rscript scripts/Candidate_genes.R